<a href="https://colab.research.google.com/github/AleksandraJuras2000/Kurs_Udemi_nn/blob/main/05_sieci_rnn/sieci_rekurencyjne_przetwarzanie_tekstu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import os

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

In [ ]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ann-course/reviews.zip
!unzip -q reviews.zip

--2025-02-26 10:38:18--  https://storage.googleapis.com/esmartdata-courses-files/ann-course/reviews.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.207, 142.250.98.207, 142.251.107.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42878657 (41M) [application/x-zip-compressed]
Saving to: ‘reviews.zip’

reviews.zip         100%[===================>]  40.89M  22.5MB/s    in 1.8s    

2025-02-26 10:38:20 (22.5 MB/s) - ‘reviews.zip’ saved [42878657/42878657]



In [ ]:
data_dir = './reviews'
train_dir = os.path.join(data_dir, 'train')

train_texts = []
train_labels = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            train_texts.append(f.read())
            f.close()
            if label_type == 'neg':
                train_labels.append(0)
            else:
                train_labels.append(1)

In [ ]:
test_dir = os.path.join(data_dir, 'test')

test_texts = []
test_labels = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            test_texts.append(f.read())
            f.close()
            if label_type == 'neg':
                test_labels.append(0)
            else:
                test_labels.append(1)

In [ ]:
train_texts[:10]

['*Warning - no plot spoilers ahead, but movie spoilers nonetheless...* My significant other rented this for me thinking it would be a terrific romance with an all-star cast. Wow - very, very wrong. This movie is an overdone, overwrought, and overly sentimental excuse to theatrically release a student film 15 years after it was shot! The copyright date on the box said 2005, yet during the very first flashback sequence I was looking at the clothes and hairdos that were supposed to be the early 1960s, and noticed that the girls especially were wearing late 80s/early 90s dresses and hairdos. It looked as if it had been shot a good 15 or 20 years before the rest of the film! I tried to convenience myself that it was a flashback, and therefore supposed to look old, but it looked WWWWWWWAAAAAAAAAAAAYYYYYYYYYYY more 80s than 60s or even 21st century trying to be 60s... then an adult coworker of the lead character turns up, and he looks just like the boy featured in the flashback sequences (ye

In [ ]:
train_labels[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
train_labels[-10:]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:

maxlen = 100   # skracamy recenzje do 100 słów
num_words = 10000    # 10000 najczęściej pojawiających się słów
embedding_dim = 100

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_texts)

In [ ]:
list(tokenizer.index_word.items())[:20]

[(1, 'the'),
 (2, 'and'),
 (3, 'a'),
 (4, 'of'),
 (5, 'to'),
 (6, 'is'),
 (7, 'br'),
 (8, 'in'),
 (9, 'it'),
 (10, 'i'),
 (11, 'this'),
 (12, 'that'),
 (13, 'was'),
 (14, 'as'),
 (15, 'for'),
 (16, 'with'),
 (17, 'movie'),
 (18, 'but'),
 (19, 'film'),
 (20, 'on')]

In [ ]:
sequences = tokenizer.texts_to_sequences(train_texts)
print(sequences[:3])

[[1706, 54, 111, 1025, 1400, 18, 17, 1025, 2932, 58, 2665, 82, 1605, 11, 15, 69, 533, 9, 59, 27, 3, 1301, 880, 16, 32, 29, 320, 174, 1314, 52, 52, 352, 11, 17, 6, 32, 3691, 8422, 2, 2078, 3169, 1335, 5, 763, 3, 1467, 19, 1114, 150, 100, 9, 13, 321, 1, 1302, 20, 1, 950, 298, 3331, 243, 312, 1, 52, 83, 2753, 717, 10, 13, 264, 30, 1, 1640, 2, 12, 68, 420, 5, 27, 1, 399, 4558, 2, 1918, 12, 1, 536, 258, 68, 1650, 519, 2079, 399, 4760, 5340, 2, 9, 605, 14, 44, 9, 66, 74, 321, 3, 49, 1114, 39, 888, 150, 156, 1, 357, 4, 1, 19, 10, 800, 5, 543, 12, 9, 13, 3, 2753, 2, 1614, 420, 5, 165, 151, 18, 9, 605, 50, 2079, 71, 3348, 39, 57, 6108, 1115, 266, 5, 27, 3348, 92, 32, 1155, 4, 1, 482, 106, 502, 53, 2, 26, 269, 40, 37, 1, 427, 2553, 8, 1, 2753, 841, 243, 42, 3, 272, 73, 919, 106, 622, 1927, 6, 2553, 8, 1, 2178, 10, 13, 337, 1468, 363, 10, 216, 8, 1, 315, 941, 1, 343, 19, 1919, 9, 13, 29, 1, 2753, 841, 321, 8, 4517, 14, 3, 597, 1467, 19, 4, 1, 169, 422, 14, 11, 17, 9, 79, 941, 1699, 12, 1679, 1, 1

In [ ]:
word_index = tokenizer.word_index
print(f'{len(word_index)} unikatowych słów.')

88582 unikatowych słów.


In [ ]:
# skracamy recenzje do pierwszych 100 słów
train_data = pad_sequences(sequences, maxlen=maxlen)
train_data.shape

(25000, 100)

In [ ]:
train_data[:3]

array([[   3, 2332,   90,   31,    1, 1718,  129,    8,    1,  399, 3129,
           5,   24,  247,   12,   33,   59,  906,   20,    1, 5975,  248,
           4,    1, 5975, 3230,    4,    1, 5975,  288,    4,    1,  159,
        8595,   32, 1335,    5, 1255,    1, 3285,  184,    1,  151,   19,
           8, 3331,  131,  159,  136,    2,  111,  235,   25,   74,  606,
         546,    1,  370, 2078, 3169,  600,   12, 7476, 3250,  117,  217,
         172,  682,  159,  133,    2,    1, 2706,  202,   12,    1,  159,
          17,   89, 1408,  222,    3,  279,  135,  871,  112,  553,    4,
          11,   17,   57,  148,    9,   45,    3,   70,  570,  174,   42,
         391],
       [1226,    1,   61,   28,   34,  691,   86,    5,  276,  852,   16,
           1,  436,    1, 1975,   12,    6, 1126,  242,   51,    1,   61,
         180,   26,   67,   27,  249,    4,    6,   12,    1, 1975,    6,
         579,   15,    1,  247,   34,    6,   40, 5558,   87,  147,    2,
          12,   26,  56

In [ ]:
train_labels = np.asarray(train_labels)
train_labels

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
# przemieszanie próbek
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)
train_data = train_data[indices]
train_labels = train_labels[indices]

train_data.shape

(25000, 100)

In [ ]:
# podział na zbiór treningowy i walidacyjny
training_samples = 15000
validation_samples = 10000

X_train = train_data[:training_samples]
y_train = train_labels[:training_samples]
X_val = train_data[training_samples: training_samples + validation_samples]
y_val = train_labels[training_samples: training_samples + validation_samples]

In [ ]:
# budowa modelu
# Embedding(input_dim, output_dim)

model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=maxlen))
model.build(input_shape=(None, maxlen))   # trzebabyło to tu dodać, Czasami TensorFlow opóźnia budowę modelu, dopóki nie zobaczy rzeczywistych danych. Możesz wymusić budowę, przekazując input_shape do pierwszej warstw
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 100, 100)            │       1,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 10000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 16)                  │         160,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,160,033 (4.43 MB)

 Trainable params: 1,160,033 (4.43 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.6703 - loss: 0.5860 - val_accuracy: 0.8411 - val_loss: 0.3627
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.9284 - loss: 0.1958 - val_accuracy: 0.8352 - val_loss: 0.3990
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9939 - loss: 0.0307 - val_accuracy: 0.8259 - val_loss: 0.5440
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9997 - loss: 0.0025 - val_accuracy: 0.8137 - val_loss: 0.7028
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9999 - loss: 6.3415e-04 - val_accuracy: 0.8172 - val_loss: 0.7612


In [ ]:
def plot_hist(history):
    import pandas as pd
    import plotly.graph_objects as go
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='accuracy', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='val_accuracy', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='accuracy vs. val accuracy', xaxis_title='Epoki', yaxis_title='accuracy', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='loss vs. val loss', xaxis_title='Epoki', yaxis_title='loss', yaxis_type='log')
    fig.show()

plot_hist(history)

In [ ]:
# jak widac powyżej model od razu się przeucza wiec próbujemy ten model poprawic i zastosujemy warstwy rekurencji
sequences = tokenizer.texts_to_sequences(test_texts)
X_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(test_labels)

model.evaluate(X_test, y_test, verbose=0)

[0.745175302028656, 0.8140000104904175]

## WARSTWY REKURENCYJNE

In [ ]:
from tensorflow.keras.layers import SimpleRNN, LSTM

In [ ]:
model = Sequential()
model.add(Embedding(10000, 32))
model.build(input_shape=(None, maxlen))
model.add(SimpleRNN(16))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (None, 100, 32)             │         320,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 16)                  │             784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,801 (1.22 MB)

 Trainable params: 320,801 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.6691 - loss: 0.6033 - val_accuracy: 0.8346 - val_loss: 0.3934
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.8592 - loss: 0.3425 - val_accuracy: 0.8328 - val_loss: 0.3877
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.8967 - loss: 0.2700 - val_accuracy: 0.8427 - val_loss: 0.3792
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9199 - loss: 0.2190 - val_accuracy: 0.8329 - val_loss: 0.3953
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9443 - loss: 0.1604 - val_accuracy: 0.8395 - val_loss: 0.4156
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9675 - loss: 0.1036 - val_accuracy: 0.8099 - val_loss: 0.4751
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9811 - loss: 0.0641 - val_accuracy: 0.8026 - val_loss: 0.5841
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9863 - loss: 0.0439 - val

In [ ]:
plot_hist(history)

In [ ]:
model = Sequential()
model.add(Embedding(10000, 32))
model.build(input_shape=(None, maxlen))
model.add(LSTM(16))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ (None, 100, 32)             │         320,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 323,153 (1.23 MB)

 Trainable params: 323,153 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 15s 29ms/step - accuracy: 0.6368 - loss: 0.6134 - val_accuracy: 0.8405 - val_loss: 0.3782
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.8606 - loss: 0.3275 - val_accuracy: 0.8509 - val_loss: 0.3474
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 20s 28ms/step - accuracy: 0.8956 - loss: 0.2621 - val_accuracy: 0.8578 - val_loss: 0.3421
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - accuracy: 0.9122 - loss: 0.2252 - val_accuracy: 0.8553 - val_loss: 0.3490
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 20s 28ms/step - accuracy: 0.9273 - loss: 0.1889 - val_accuracy: 0.8431 - val_loss: 0.4032
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - accuracy: 0.9373 - loss: 0.1693 - val_accuracy: 0.8492 - val_loss: 0.4044
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.9438 - loss: 0.1553 - val_accuracy: 0.8367 - val_loss: 0.4332
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 20s 28ms/step - accuracy: 0.9498 - loss: 0.1420 - 

In [ ]:
plot_hist(history)

Sprawdzenie na danych testowych

In [ ]:
model = Sequential()
model.add(Embedding(10000, 32))
model.build(input_shape=(None, maxlen))
model.add(LSTM(16))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=32, epochs=3, validation_data=(X_val, y_val))

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ (None, 100, 32)             │         320,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 16)                  │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 323,153 (1.23 MB)

 Trainable params: 323,153 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 15s 29ms/step - accuracy: 0.6196 - loss: 0.6290 - val_accuracy: 0.8254 - val_loss: 0.3988
Epoch 2/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.8624 - loss: 0.3384 - val_accuracy: 0.8122 - val_loss: 0.4419
Epoch 3/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.8941 - loss: 0.2651 - val_accuracy: 0.8454 - val_loss: 0.4105


In [ ]:
sequences = tokenizer.texts_to_sequences(test_texts)   # musimy nasz zbior testowy przygotować
X_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(test_labels)

model.evaluate(X_test, y_test, verbose=0)

[0.42317426204681396, 0.8392800092697144]